<a href="https://colab.research.google.com/github/ShahadAnazi/Week9/blob/main/Blockchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
!pip install jason

In [58]:
from hashlib import sha256
import time 
import json

In [59]:
class MyBlock():
    def __init__(self,
                index = 0,
                transactions:list = [],
                timestamp:int = 0,
                prop_hash:str = "",
                prev_hash:str = "",
                nonce:int = 0):
        self.index:int = index
        self.transactions:list = transactions
        self.timestamp:int = timestamp
        self.prop_hash:str = prop_hash
        self.prev_hash:str = prev_hash
        self.nonce:int = nonce
            
            
    def compute_hash(self):
        block_string = json.dumps(self.__dict__, sort_keys=True)
        return sha256(block_string.encode()).hexdigest()

In [44]:
time.asctime()

'Sun Dec 12 05:47:06 2021'

In [60]:
class Blockchain(): 
    def __init__(self):
        self.unconfirmed_transactions:list[dict] = []
        self.chain:list[Block] = []
        self.difficulty:int=2
        self.create_genesis_block()
        
    def get_last_block(self):
        return self.chain[-1]
 
    def create_genesis_block(self):
        genesis_block = MyBlock(index=0,
                                transactions=[],
                                timestamp=time.asctime(),
                                prev_hash="",
                                prop_hash="", nonce=0)
        genesis_block.prop_hash = genesis_block.compute_hash()
        self.chain.append(genesis_block)
    
    def proof_of_work(self, block):
        
        computed_hash = block.compute_hash()
        while not computed_hash.startswith('0' * self.difficulty):
            block.nonce += 1
            computed_hash = block.compute_hash()
        return computed_hash
    
    def is_valid_proof(self, block, block_hash):
        return (block_hash.startswith('0' * self.difficulty) and
                block_hash == block.compute_hash())
    
    def add_block(self, block:MyBlock, block_hash:str) -> bool:
        last_block = self.get_last_block()
        
        if last_block.prop_hash != block.prev_hash:
            return False
        
        if not self.is_valid_proof(block, block_hash):
            return False
        
        block.prop_hash = block_hash
        
        self.chain.append(block)
        
        return True
    
    def add_new_transaction(self, transaction:dict) -> None:
            self.unconfirmed_transactions.append(transaction)
            
    def mine(self) -> int:
        if len(self.unconfirmed_transactions)<1:
            return -1
        
        last_block = self.get_last_block()
 
        new_block = MyBlock(index=last_block.index+1,
                          transactions=self.unconfirmed_transactions,
                          timestamp=(time.asctime()),
                          prev_hash=last_block.prop_hash)
    
        prop_hash = self.proof_of_work(new_block)
        if not self.add_block(new_block, prop_hash):
            return -1
        self.add_block(new_block, prop_hash)
         #new_block.hash = prop_hash
        self.unconfirmed_transactions = []
        
        
        return new_block.index

In [61]:
mb = MyBlock()
mb.__dict__ # what the block contains  

{'index': 0,
 'nonce': 0,
 'prev_hash': '',
 'prop_hash': '',
 'timestamp': 0,
 'transactions': []}

In [47]:
mb.compute_hash()

'16102a9b51bec6785fe8cb8fc35d2209e8d7dd7b873ef8cadea142f4020a8f19'

In [62]:
mb.prop_hash=mb.compute_hash()
mb

In [49]:
mb.compute_hash()

'3798682d03b9e04cea5c5e7fed2651893a51cdf0fd44bbdbd448f87381d23565'

In [50]:
mb.compute_hash()

'3798682d03b9e04cea5c5e7fed2651893a51cdf0fd44bbdbd448f87381d23565'

In [51]:
mb.compute_hash().startswith('0'*2)

False

In [52]:
bc = Blockchain()
bc.add_new_transaction({'Name': "Shahad ",'Major':"MIS"})
#x.compute_hash # calling the method of hash returning the hex digest of the block_string 

In [53]:
bc.__dict__

{'chain': [<__main__.MyBlock at 0x7f5ac34aa410>],
 'difficulty': 2,
 'unconfirmed_transactions': [{'Major': 'MIS', 'Name': 'Shahad '}]}

In [54]:
bc.unconfirmed_transactions

[{'Major': 'MIS', 'Name': 'Shahad '}]

In [55]:
bc.chain[0].__dict__

{'index': 0,
 'nonce': 0,
 'prev_hash': '',
 'prop_hash': 'ad19e35b9066247f41193422467a34f3d31e404b526b62a5330350989e869d52',
 'timestamp': 'Sun Dec 12 05:47:06 2021',
 'transactions': []}

In [56]:
bc.mine()

1

In [63]:
bc.add_new_transaction({"addr_from":"ad19e35b9066247f41193422467a34f3d31e404b526b62a5330350989e869d52",
                       "addr_to": "140e1a639663cc35cc991efd53e9b32b3b705bee3479b1036b921d986d5cf01b",
                       "amount": 99.00})

In [64]:
bc.__dict__

{'chain': [<__main__.MyBlock at 0x7f5ac34aa410>,
 'difficulty': 2,
 'unconfirmed_transactions': [{'addr_from': 'ad19e35b9066247f41193422467a34f3d31e404b526b62a5330350989e869d52',
   'addr_to': '140e1a639663cc35cc991efd53e9b32b3b705bee3479b1036b921d986d5cf01b',
   'amount': 99.0}]}

In [65]:
bc.unconfirmed_transactions

[{'addr_from': 'ad19e35b9066247f41193422467a34f3d31e404b526b62a5330350989e869d52',
  'addr_to': '140e1a639663cc35cc991efd53e9b32b3b705bee3479b1036b921d986d5cf01b',
  'amount': 99.0}]

In [66]:
bc.chain[1].__dict__

{'index': 1,
 'nonce': 572,
 'prev_hash': 'ad19e35b9066247f41193422467a34f3d31e404b526b62a5330350989e869d52',
 'prop_hash': '00f5fa14dd766b53f0a008b122d2825e4b1af4397d37bcb7c74309d5da5e21e0',
 'timestamp': 'Sun Dec 12 05:47:06 2021',
 'transactions': [{'Major': 'MIS', 'Name': 'Shahad '}]}

In [67]:
bc.mine()

2

In [68]:
bc.get_last_block()